In [1]:
from NewLibraryENG import *

In [2]:
import glob
list_dir=glob.glob("/Users/Gaia_1/Desktop/h5files/*.h5")

In [ ]:
fs = 10000 #Sampling Frequency
all_data=[]
all_ref=[]
for file in tqdm(list_dir):
    string=file
    data=h5py.File(string,'r')
    data_readings = data['Data']['Recording_0']['AnalogStream']['Stream_0']['ChannelData'][()]
    info = data['Data']['Recording_0']['AnalogStream']['Stream_0']['InfoChannel'][()]
    info_table = pd.DataFrame(info, columns = list(info.dtype.fields.keys()))
    labels = info_table['Label']
    readings = pd.DataFrame(data = data_readings.transpose(), columns = labels)
    ref=readings[b'Ref']
    len_data=int(len(readings)/6)
    readings=readings.drop([b'Ref'],axis=1)
    readings=readings.iloc[0:len_data, 17:23]
    ref=ref[0:len_data]
    all_ref.append(ref)
    print(readings.shape)
    all_data.append(readings)

In [ ]:
lowcut = 300
highcut = 3000
fs=10000
order=8
b,a=butter_bandpass(lowcut,highcut,fs,order=order)
filtered_all_data=[]
for file in tqdm(range(len(all_data))):
    prova=all_data[file]
    ref=all_ref[file]
    prova_rows = range(prova.shape[0])
    filt_prova = pd.DataFrame(data = 0, columns=prova.columns, index=prova_rows, dtype = "float32")
    filt_ref=filtfilt(b,a,ref)
    for x in tqdm(range(prova.shape[1])):
        filt_prova.values[:,x] = scipy.signal.filtfilt(b, a, prova.values[:,x])
    for electrode in prova.columns:
        filt_prova[electrode] = filt_prova[electrode] - filt_ref
    filtered_all_data.append(filt_prova)
    filt_prova.shape

# Detection

In [ ]:
pos_ind=[]
neg_ind=[]
for file in tqdm(range(len(filtered_all_data))):
    pos_ind_file=[]
    neg_ind_file=[]
    prova=filtered_all_data[file]
    for electrode in tqdm(prova.columns):
        channel=prova[electrode]
        pos, neg=find_all_spikes(channel)
        pos_ind_file.append(pos)
        neg_ind_file.append(neg)
    pos_ind.append(pos_ind_file)
    neg_ind.append(neg_ind_file)

# Cutouts

In [ ]:
pos_cut=[]
neg_cut=[]
n_pos=[]
n_neg=[]
for file in tqdm(range(len(filtered_all_data))):
    pos_cut_file=[]
    neg_cut_file=[]
    n_pos_file=[]
    n_neg_file=[]
    prova=filtered_all_data[file]
    pos_ind_file=pos_ind[file]
    neg_ind_file=neg_ind[file]
    for i,electrode in enumerate(tqdm(prova.columns)):
        pos=pos_ind_file[i]
        neg=neg_ind_file[i]
        channel=prova[electrode]
        pos_cut1,n_pos1, neg_cut1,n_neg1 = cut(pos,neg,channel)
        pos_cut_file.append(pos_cut1)
        neg_cut_file.append(neg_cut1)
        n_pos_file.append(n_pos1)
        n_neg_file.append(n_neg1)
    pos_cut.append(pos_cut_file)
    neg_cut.append(neg_cut_file)
    n_pos.append(n_pos_file)
    n_neg.append(n_neg_file)
#savedp = copy.deepcopy(pos_cut)
#savedn = copy.deepcopy(neg_cut)

In [ ]:
for file in tqdm(range(len(filtered_all_data))):
    pos_cut_file=pos_cut[file]
    neg_cut_file=neg_cut[file]
    for i in (tqdm(range(len(pos_cut_file)))):
        pos_cut_file[i]=mask_cuts(pos_cut_file[i])
        neg_cut_file[i]= mask_cuts(neg_cut_file[i])
    pos_cut[file]=pos_cut_file
    neg_cut[file]=neg_cut_file

# Clustering

In [ ]:
pos_clusters=[]
for file in tqdm(range(len(filtered_all_data))):
    data=filtered_all_data[file]
    pos_cut_file=pos_cut[file]
    n_pos_file=n_pos[file]
    final_data_pos=[]
    for channel in (tqdm(range(len(pos_cut_file)))):
        channel_clusters=clus(pos_cut_file[channel],'kmeans',n_pos_file[channel],data)
        final_data_pos.append(channel_clusters)
    pos_clusters.append(final_data_pos)

In [ ]:
neg_clusters=[]
for file in tqdm(range(len(filtered_all_data))):
    data=filtered_all_data[file]
    neg_cut_file=neg_cut[file]
    n_neg_file=n_neg[file]
    final_data_neg=[]
    for channel in (tqdm(range(len(neg_cut_file)))):
        channel_clusters=clus(neg_cut_file[channel],'kmeans',n_neg_file[channel],data)
        final_data_neg.append(channel_clusters)
    neg_clusters.append(final_data_neg)

In [ ]:
### verificare differenze tra kmeans e fuzzy c means

In [ ]:
neurons=[]
for file in pos_clusters:
    neurons_f=[]
    for channel in file:
        neurons_c=[]
        for neuron in channel:
            neurons_c.append(neuron)
        neurons_f.append(neurons_c)
    neurons.append(neurons_f)
len(neurons)

In [ ]:
len(neurons[0][0])

In [ ]:
neurons[0][0]

In [ ]:
neurons[0][0]

In [ ]:
for file in neg_clusters:
    neurons_f=[]
    for channel in file:
        neurons_c=[]
        for neuron in channel:
            neurons_c.append(neuron)
        neurons_f.append(neurons_c)
    neurons.append(neurons_f)
len(neurons)

In [ ]:
len(neurons[0][3])

In [ ]:
adj_neur=[]
counter = 0
max_len=0

for file in neurons:
    for clus in file:
        for neu in clus:
            print('counter: ',counter,neu.shape[0])
            if neu.shape[0]>max_len:
                max_len=neu.shape[0]
            counter+=1
    counter=0
print('max_len',max_len)

In [ ]:
for file in neurons:
    for clus in file:
        adj_neur_clus=[]
        for neu in clus:
            for s in neu:
                print(s)

In [ ]:
for file in neurons:
    for clus in file:
        adj_neur_clus=[]
        for neu in clus:
            same_length_spike=[]
            if neu.shape[0]<=max_len:
                print(len(neu))
                diff = max_len-neu.shape[0]
                same_length_spike.append(np.concatenate((neu,np.zeros([diff]))))
                print(same_length_spike)
        adj_neur_clus.append(same_length_spike)
    adj_neur.append(adj_neur_clus)
len(adj_neur)

In [ ]:
adj_neur[4]

# Export (after sorting)

In [ ]:
import os
for i,file in enumerate(list_dir):
    name_data=os.path.basename(list_dir[i])
    save_data = 'After'+name_data
    #np.savetxt(r"C:\Data after SS\%save_data.txt" % save_data,adj_neur[i], delimiter=', ', fmt='%12.8f')
    np.savetxt("/Users/Gaia_1/Desktop/tesi/Data after SS/%s.txt" % save_data,adj_neur[i], delimiter=', ', fmt='%12.8f')
    

# Point Process

In [3]:
def Bayesian_mixture_model(ISI_data):
    with pm.Model() as model:
        ##### WALD DISTRIBUTION (INVERSE GAUSSIAN)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0,upper=0.2)
        sigma2 = pm.Uniform('sigma2',lower=0.0001,upper=0.5)
        obs2 = pm.TruncatedNormal.dist(mu=mu2, sigma=sigma2, lower=0.0)

        mu3 = pm.Uniform('mu3',lower=0.1,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.0001,upper=0.5)
        obs3 = pm.TruncatedNormal.dist(mu=mu3, sigma=sigma3, lower=0.0)


        w = pm.Dirichlet('w', a=np.array([1., .4, .4]))
        #w = pm.Dirichlet('w', a=np.array([1., .4]))

        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)
        #like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2], observed=ISI_data)

        step = pm.NUTS(target_accept=0.9)
        trace = pm.sample(step=step,draws=1000,chains=1,tune=1000,cores=4)
        
        #ppc_trace = pm.sample_posterior_predictive(trace,model=model)
        
    map_estimate = pm.find_MAP(model=model)
    
    del map_estimate['w_simplex__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma2_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']
    
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]

    del map_estimate['w']


    return map_estimate#, ppc_trace


In [7]:
list_dir=glob.glob("Data after SS/*.txt")

In [8]:
list_dir_ok = []
for net in list_dir:
    #if 'After' not in net:
        print(net,'ok')
        list_dir_ok.append(net)

Data after SS/After2018-11-27T10-56-39MiP5 KA stimulation.h5.txt.txt ok
Data after SS/After2018-11-27T10-29-42POP 2 BL .h5.txt.txt ok
Data after SS/After2019-01-23T11-05-52MIP3 health cortical .h5.txt.txt ok
Data after SS/After2018-11-27T11-24-28MiP3 stimulation.h5.txt.txt ok
Data after SS/After2018-11-27T10-40-53POP 3 BL .h5.txt.txt ok
Data after SS/After2018-11-27T12-03-02POP 2 stimulation.h5.txt.txt ok


In [9]:
n_neurons_healthy = 0
n_neurons_lrrk2 = 0
for net in list_dir_ok:
    list_neurons = np.genfromtxt(net, delimiter=',')
    if 'health' in net:
            print('H',net,list_neurons.shape[0])
            n_neurons_healthy = n_neurons_healthy+list_neurons.shape[0]
            
    elif 'Healthy' in net:
            print('H',net,list_neurons.shape[0])
            n_neurons_healthy = n_neurons_healthy+list_neurons.shape[0]
    else:
            print('L',net,list_neurons.shape[0])
            n_neurons_lrrk2 = n_neurons_lrrk2+list_neurons.shape[0]
print('Total healthy neurons: ',n_neurons_healthy,'\n')
print('Total lrrk2 neurons: ',n_neurons_lrrk2,'\n')

L Data after SS/After2018-11-27T10-56-39MiP5 KA stimulation.h5.txt.txt 121
L Data after SS/After2018-11-27T10-29-42POP 2 BL .h5.txt.txt 94
H Data after SS/After2019-01-23T11-05-52MIP3 health cortical .h5.txt.txt 99
L Data after SS/After2018-11-27T11-24-28MiP3 stimulation.h5.txt.txt 106
L Data after SS/After2018-11-27T10-40-53POP 3 BL .h5.txt.txt 97
L Data after SS/After2018-11-27T12-03-02POP 2 stimulation.h5.txt.txt 80
Total healthy neurons:  99 

Total lrrk2 neurons:  498 



In [10]:
for net in list_dir_ok:
    print(net)
    list_neurons = np.genfromtxt(net, delimiter=',')
    print(list_neurons)

Data after SS/After2018-11-27T10-56-39MiP5 KA stimulation.h5.txt.txt
[[ 2296.  4207. 11849. ...     0.     0.     0.]
 [ 1537.  3678.  4566. ...     0.     0.     0.]
 [ 3172.  4145.  4945. ...     0.     0.     0.]
 ...
 [ 1962.  5349.  5701. ...     0.     0.     0.]
 [ 6880.  9642. 10934. ...     0.     0.     0.]
 [12643. 12655. 14879. ...     0.     0.     0.]]
Data after SS/After2018-11-27T10-29-42POP 2 BL .h5.txt.txt
[[ 5121. 13902. 14871. ...     0.     0.     0.]
 [  171.  4801. 12018. ...     0.     0.     0.]
 [ 2280.  7259. 17159. ...     0.     0.     0.]
 ...
 [ 3387. 13107. 25496. ...     0.     0.     0.]
 [ 4195.  7621.  8573. ...     0.     0.     0.]
 [ 7735. 14037. 21988. ...     0.     0.     0.]]
Data after SS/After2019-01-23T11-05-52MIP3 health cortical .h5.txt.txt
[[   3458.    3951.    4649. ... 2241736. 2241841. 2247664.]
 [   5819.    9848.   17820. ...       0.       0.       0.]
 [   3963.    7689.    8086. ...       0.       0.       0.]
 ...
 [   5415.   

In [11]:
a=np.genfromtxt(list_dir_ok[0], delimiter=',')
a

array([[ 2296.,  4207., 11849., ...,     0.,     0.,     0.],
       [ 1537.,  3678.,  4566., ...,     0.,     0.,     0.],
       [ 3172.,  4145.,  4945., ...,     0.,     0.,     0.],
       ...,
       [ 1962.,  5349.,  5701., ...,     0.,     0.,     0.],
       [ 6880.,  9642., 10934., ...,     0.,     0.,     0.],
       [12643., 12655., 14879., ...,     0.,     0.,     0.]])

In [12]:
b=adj_neur[0]
len(b[0])

NameError: name 'adj_neur' is not defined

In [13]:
dataframe = pd.DataFrame()
counter_net=1
counter=0
target = 0
for net in tqdm(list_dir_ok):
    print(counter_net,') ',net)
    counter_net+=1
    list_neurons = np.genfromtxt(net, delimiter=',')
    counter=0
    print('Original number of neurons: ',len(list_neurons))
    for neuron in list_neurons:
        neuron=neuron[neuron>0*10000]
        neuron=neuron[neuron<200*10000]
        print('  Neuron with ',neuron.shape[0],'spikes')
        if neuron.shape[0]>1000:
            
            counter+=1
        else:
            print('    Excluded neuron with n spikes = ',neuron.shape[0])
            continue
        
        ISI_healthy = np.diff(neuron)/10000
        
            
        map_estimate = Bayesian_mixture_model(ISI_healthy)
        
        if 'Healthy' in net:
            map_estimate['Target']=0
            
        elif 'Healthy' in net:
            map_estimate['Target']=0
        else:
            map_estimate['Target']=1
        
        if 'KA' in net:
            map_estimate['Stimulation']=1
            
        elif 'stimulation' in net:
            map_estimate['Stimulation']=1
        else:
            map_estimate['Stimulation']=0
            
        df = pd.DataFrame.from_dict(map_estimate,orient='index')
        dataframe = pd.concat([dataframe,df],axis = 1)
    print('Final number of neurons: ',counter)
    print('Target = ',target)

1 )  Data after SS/After2018-11-27T10-56-39MiP5 KA stimulation.h5.txt.txt
Original number of neurons:  121
  Neuron with  359 spikes
    Excluded neuron with n spikes =  359
  Neuron with  751 spikes
    Excluded neuron with n spikes =  751
  Neuron with  362 spikes
    Excluded neuron with n spikes =  362
  Neuron with  411 spikes
    Excluded neuron with n spikes =  411
  Neuron with  605 spikes
    Excluded neuron with n spikes =  605
  Neuron with  521 spikes
    Excluded neuron with n spikes =  521
  Neuron with  376 spikes
    Excluded neuron with n spikes =  376
  Neuron with  417 spikes
    Excluded neuron with n spikes =  417
  Neuron with  287 spikes
    Excluded neuron with n spikes =  287
  Neuron with  353 spikes
    Excluded neuron with n spikes =  353
  Neuron with  440 spikes
    Excluded neuron with n spikes =  440
  Neuron with  379 spikes
    Excluded neuron with n spikes =  379
  Neuron with  337 spikes
    Excluded neuron with n spikes =  337
  Neuron with  221 spi

Sequential sampling (1 chains in 1 job)
NUTS: [mu1, lam1, mu2, sigma2, mu3, sigma3, w]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 66 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks



  Neuron with  1279 spikes


Sequential sampling (1 chains in 1 job)
NUTS: [mu1, lam1, mu2, sigma2, mu3, sigma3, w]


...

Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 64 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks



  Neuron with  804 spikes
    Excluded neuron with n spikes =  804
  Neuron with  405 spikes
    Excluded neuron with n spikes =  405
  Neuron with  310 spikes
    Excluded neuron with n spikes =  310
  Neuron with  266 spikes
    Excluded neuron with n spikes =  266
  Neuron with  346 spikes
    Excluded neuron with n spikes =  346
  Neuron with  321 spikes
    Excluded neuron with n spikes =  321
  Neuron with  336 spikes
    Excluded neuron with n spikes =  336
  Neuron with  227 spikes
    Excluded neuron with n spikes =  227
  Neuron with  231 spikes
    Excluded neuron with n spikes =  231
  Neuron with  254 spikes
    Excluded neuron with n spikes =  254
  Neuron with  402 spikes
    Excluded neuron with n spikes =  402
  Neuron with  270 spikes
    Excluded neuron with n spikes =  270
  Neuron with  405 spikes
    Excluded neuron with n spikes =  405
  Neuron with  310 spikes
    Excluded neuron with n spikes =  310
  Neuron with  266 spikes
    Excluded neuron with n spikes =

Sequential sampling (1 chains in 1 job)
NUTS: [mu1, lam1, mu2, sigma2, mu3, sigma3, w]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 50 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks



  Neuron with  339 spikes
    Excluded neuron with n spikes =  339
  Neuron with  484 spikes
    Excluded neuron with n spikes =  484
  Neuron with  640 spikes
    Excluded neuron with n spikes =  640
  Neuron with  356 spikes
    Excluded neuron with n spikes =  356
  Neuron with  302 spikes
    Excluded neuron with n spikes =  302
  Neuron with  692 spikes
    Excluded neuron with n spikes =  692
  Neuron with  461 spikes
    Excluded neuron with n spikes =  461
  Neuron with  873 spikes
    Excluded neuron with n spikes =  873
  Neuron with  337 spikes
    Excluded neuron with n spikes =  337
  Neuron with  510 spikes
    Excluded neuron with n spikes =  510
  Neuron with  602 spikes
    Excluded neuron with n spikes =  602
  Neuron with  354 spikes
    Excluded neuron with n spikes =  354
  Neuron with  395 spikes
    Excluded neuron with n spikes =  395
  Neuron with  437 spikes
    Excluded neuron with n spikes =  437
  Neuron with  640 spikes
    Excluded neuron with n spikes =

In [14]:
final = dataframe.T
final.to_csv('Data after PP/DataAfter')

# Classification

In [16]:
dataset = pd.read_csv('Data after PP/DataAfter')

In [18]:
dataset = dataset.drop(['Unnamed: 0'],axis=1)

In [19]:
dataset

,mu1,lam1,mu2,sigma2,mu3,sigma3,w1,w2,w3,Target,Stimulation
0,0.100000,0.040000,2.022434e-07,0.125512,0.100000,0.276938,0.198715,0.451119,0.350167,1.0,1.0
1,0.079097,0.017746,3.477574e-02,0.125393,0.100018,0.301054,0.193046,0.511017,0.295937,1.0,1.0
2,0.100000,0.019735,1.899310e-14,0.135923,0.100000,0.326558,0.073456,0.513832,0.412712,1.0,1.0
3,0.100000,0.010000,4.961221e-16,0.116094,0.100000,0.301565,0.039970,0.481280,0.478750,1.0,1.0
4,0.084294,0.026168,6.424867e-15,0.117040,0.100002,0.238710,0.213444,0.530212,0.256344,1.0,1.0
5,0.081458,0.040000,1.271153e-13,0.192667,0.100003,0.399239,0.183813,0.640335,0.175852,1.0,0.0


In [21]:
from sklearn.model_selection import train_test_split
X = dataset.drop(['Target'],axis=1)
y= dataset['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    stratify=y,
                                                    random_state=42)

In [22]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(4)

In [26]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3,svd_solver='full')
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print(pca.fit(X_train).explained_variance_ratio_)

[0.52918363 0.31961353 0.15120284]


Decision Tree

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
params = {'max_depth':np.linspace(2,10,dtype='int'),'criterion':['gini','entropy']}

clf = DecisionTreeClassifier(random_state=12)

gs = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,n_jobs=-1)

gs=gs.fit(X_train,y_train)

gs_pca = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,n_jobs=-1)
gs_pca = gs_pca.fit(X_train_pca,y_train)

/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 459, in _score
    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 181, in _get_response_values
    prediction_method = _check_response_method(estimator, response_method)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_

/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 459, in _score
    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 181, in _get_response_values
    prediction_method = _check_response_method(estimator, response_method)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_

    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 181, in _get_response_values
    prediction_method = _check_response_method(estimator, response_method)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py", line 1939, in _check_response_method
    raise AttributeError(
AttributeError: DecisionTreeClassifier has none of the following attributes: decision_function.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Gaia_1/anac

    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 466, in _score
    y_pred = method_caller(clf, "predict_proba", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 197, in _get_response_values
    y_pred = _process_predict_proba(
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 50, in _process_predict_proba
    raise ValueError(
ValueError: Got predict_proba of shape

    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 466, in _score
    y_pred = method_caller(clf, "predict_proba", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 197, in _get_response_values
    y_pred = _process_predict_proba(
             ^^^

...

  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 197, in _get_response_values
    y_pred = _process_predict_proba(
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 50, in _process_predict_proba
    raise ValueError(
ValueError: Got predict_proba of shape (1, 1), but need classifier with two classes.

  warnings.warn(
/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 459, in _score
    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/p

             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 181, in _get_response_values
    prediction_method = _check_response_method(estimator, response_method)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py", line 1939, in _check_response_method
    raise AttributeError(
AttributeError: DecisionTreeClassifier has none of the following attributes: decision_function.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", l

    y_pred = method_caller(clf, "predict_proba", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 197, in _get_response_values
    y_pred = _process_predict_proba(
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 50, in _process_predict_proba
    raise ValueError(
ValueError: Got predict_proba of shape (1, 1), but need classifier with two classes.

  warnings.warn(
/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 

                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 197, in _get_response_values
    y_pred = _process_predict_proba(
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 50, in _process_predict_proba
    raise ValueError(
ValueError: Got predict_proba of shape (1, 1), but need classifier with two classes.

  warnings.warn(
/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 459, in _score
    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 459, in _score
    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/_response.py", line 181, in _get_response_values
    prediction_method = _check_response_method(estimator, response_method)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/Gaia_1/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py", line 1939, in _check_response_method
    raise AttributeError(
AttributeError: DecisionTreeClassifier has none of the following attributes: decision_function.

During handling of t

In [36]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
print("Best parameters set found :",gs.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs.predict(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
#roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
#print('ROC AUC: ',roc_auc)
print('\n\n Result with PCA\n')
print("Best parameters set found :",gs_pca.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs_pca.predict(X_test_pca)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
#roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
#print('ROC AUC: ',roc_auc)

Best parameters set found : {'criterion': 'gini', 'max_depth': 2} 

Classification report on Test set

Accuracy:  1.0
Recall:  1.0
Precision:  1.0


 Result with PCA

Best parameters set found : {'criterion': 'gini', 'max_depth': 2} 

Classification report on Test set

Accuracy:  1.0
Recall:  1.0
Precision:  1.0


Logistic Regression

In [ ]:
params = {'C':np.logspace(-3,3,50),'penalty':['l1','l2','elasticnet']}

clf = LogisticRegression(solver='liblinear',random_state=12)

gs = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,n_jobs=-1)

gs=gs.fit(X_train,y_train)

In [ ]:
print("Best parameters set found :",gs.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs.predict(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)

Random Forest

In [ ]:
params = {'n_estimators':np.linspace(10,150,dtype='int',num=10),'max_depth':np.linspace(2,5,dtype='int'),'criterion':['gini','entropy']}

clf = RandomForestClassifier(n_jobs=-1,random_state=12)

gs = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,verbose=10,n_jobs=-1)

gs=gs.fit(X_train,y_train)

In [ ]:
print("Best parameters set found :",gs.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs.predict(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)

Neural Net

In [ ]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            20, activation="relu", input_shape=(X_train.shape[1],)
        ),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(50, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

In [ ]:
metrics = [
    'accuracy',
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)



history=model.fit(
    X_train,
    y_train,
    batch_size=10,
    epochs=200,
    validation_data=(X_test, y_test),

)

In [ ]:
# list all data in history
print(history.history.keys())
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_true, y_pred = y_test, model.predict_classes(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)